In [2]:
import pandas as pd

# Чтение тестового набора данных из CSV-файла
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')


In [3]:
X_train.head()

,LOCATION_ID,PARA_A,PARA_B,TOTAL,numbers,Money_Value,Loss,History
0,4,0.28,0.25,0.53,5.0,0.00,0,0
1,31,0.00,0.00,0.00,5.0,0.03,0,0
2,4,5.65,67.16,72.81,6.0,27.23,0,0
3,11,1.27,0.00,1.27,5.0,175.90,0,0
4,19,1.60,0.00,1.60,5.0,0.06,0,0


# 23

In [4]:
X_train.nunique()

LOCATION_ID     40
PARA_A         287
PARA_B         262
TOTAL          356
numbers          5
Money_Value    247
Loss             3
History          7
dtype: int64

# 24

In [5]:
# Проверяем, есть ли нечисловые значения в столбце LOCATION_ID
numeric_mask = pd.to_numeric(X_train['LOCATION_ID'], errors='coerce').notna()

# Фильтруем только строки с числовыми значениями в LOCATION_ID
X_train = X_train[numeric_mask]

# Удаляем соответствующие строки из y_train
y_train = y_train[numeric_mask].reset_index(drop=True)

# Сбрасываем индексы в X_train
X_train = X_train.reset_index(drop=True)

# Выводим новое количество строк в таблице X_train
print(f"Новое количество строк в таблице X_train: {len(X_train)}")


Новое количество строк в таблице X_train: 541


# 25

In [6]:
# Проверяем наличие пропущенных значений в тренировочных данных
missing_values = X_train.isnull().sum()
print("Количество пропущенных значений в каждом столбце:")
print(missing_values)

# Подсчитываем количество строк с пропущенными значениями
rows_with_missing = X_train.isnull().any(axis=1).sum()
print(f"\nКоличество строк, содержащих пропущенные значения: {rows_with_missing}")


Количество пропущенных значений в каждом столбце:
LOCATION_ID    0
PARA_A         0
PARA_B         0
TOTAL          0
numbers        0
Money_Value    1
Loss           0
History        0
dtype: int64

Количество строк, содержащих пропущенные значения: 1


In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541 entries, 0 to 540
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   LOCATION_ID  541 non-null    object 
 1   PARA_A       541 non-null    float64
 2   PARA_B       541 non-null    float64
 3   TOTAL        541 non-null    float64
 4   numbers      541 non-null    float64
 5   Money_Value  540 non-null    float64
 6   Loss         541 non-null    int64  
 7   History      541 non-null    int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 33.9+ KB


# 26

In [8]:
# Заменяем пропущенные значения в столбце Money_Value на среднее значение
mean_money_value = X_train['Money_Value'].mean()
X_train['Money_Value'].fillna(mean_money_value, inplace=True)

# Проверяем, что пропуски были заменены
print(f"Количество пропущенных значений в столбце Money_Value после замены: {X_train['Money_Value'].isnull().sum()}")
print(f"Среднее значение, использованное для замены: {mean_money_value:.2f}")


Количество пропущенных значений в столбце Money_Value после замены: 0
Среднее значение, использованное для замены: 16.45


/var/folders/f2/942c2cl91r9877yf8jj8c15xr94h9h/T/ipykernel_69025/3651075081.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Money_Value'].fillna(mean_money_value, inplace=True)


In [9]:
# Проверяем наличие пропущенных значений в тренировочных данных
missing_values = X_test.isnull().sum()
print("Количество пропущенных значений в каждом столбце:")
print(missing_values)

# Подсчитываем количество строк с пропущенными значениями
rows_with_missing = X_test.isnull().any(axis=1).sum()
print(f"\nКоличество строк, содержащих пропущенные значения: {rows_with_missing}")


Количество пропущенных значений в каждом столбце:
LOCATION_ID    0
PARA_A         0
PARA_B         0
TOTAL          0
numbers        0
Money_Value    0
Loss           0
History        0
dtype: int64

Количество строк, содержащих пропущенные значения: 0


# 27

In [10]:
# Подсчитываем частоту встречаемости каждого значения LOCATION_ID в X_train
location_counts = X_train['LOCATION_ID'].value_counts()

# Создаем словарь для маппинга значений
mapping = {}
for location, count in location_counts.items():
    # Определяем букву на основе количества
    letter_index = (count - 1) // 10  # 0-9 -> 0, 10-19 -> 1, 20-29 -> 2, etc.
    letter = chr(65 + letter_index)  # 65 это ASCII код 'A'
    mapping[location] = letter

# Применяем маппинг к X_train
X_train['LOCATION_ID'] = X_train['LOCATION_ID'].map(mapping)

# Применяем маппинг к X_test с дефолтным значением 'A' для новых категорий
X_test['LOCATION_ID'] = X_test['LOCATION_ID'].map(lambda x: mapping.get(x, 'A'))

# Проверяем количество уникальных категорий в X_train
n_categories = X_train['LOCATION_ID'].nunique()
print(f"Количество различных категорий в X_train: {n_categories}")

# Проверяем совпадение категорий
train_categories = set(X_train['LOCATION_ID'].unique())
test_categories = set(X_test['LOCATION_ID'].unique())
print("\nКатегории в X_train:", sorted(train_categories))
print("Категории в X_test:", sorted(test_categories))
print("\nКатегории совпадают:", train_categories == test_categories)

Количество различных категорий в X_train: 5

Категории в X_train: ['A', 'B', 'C', 'D', 'F']
Категории в X_test: ['A']

Категории совпадают: False


# 28

In [11]:
# Сначала посмотрим название столбца в y_train
print("Столбцы в y_train:", y_train.columns.tolist())

# Получаем список числовых признаков (исключая LOCATION_ID)
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns

# Вычисляем корреляционную матрицу
corr_matrix = X_train[numeric_features].corr()

# Находим пары признаков с корреляцией > 0.9 по модулю
high_corr_pairs = []
for i in range(len(numeric_features)):
    for j in range(i + 1, len(numeric_features)):
        if abs(corr_matrix.iloc[i, j]) > 0.9:
            feature1 = numeric_features[i]
            feature2 = numeric_features[j]
            high_corr_pairs.append((feature1, feature2))

# Вычисляем корреляцию с целевой переменной
# Используем первый столбец y_train, так как обычно он и есть целевая переменная
target = y_train.iloc[:, 0]
target_corr = abs(X_train[numeric_features].corrwith(target))

# Определяем, какие признаки нужно удалить
features_to_drop = set()
for feat1, feat2 in high_corr_pairs:
    # Удаляем признак с меньшей корреляцией с целевой переменной
    if target_corr[feat1] < target_corr[feat2]:
        features_to_drop.add(feat1)
    else:
        features_to_drop.add(feat2)

# Выводим информацию о найденных парах и удаляемых признаках
print("\nПары признаков с высокой корреляцией:")
for feat1, feat2 in high_corr_pairs:
    print(f"{feat1} - {feat2}: {corr_matrix.loc[feat1, feat2]:.3f}")

print("\nПризнаки, которые будут удалены:")
for feat in features_to_drop:
    print(f"{feat} (корреляция с целевой: {target_corr[feat]:.3f})")

# Удаляем выбранные признаки из обоих наборов данных
X_train = X_train.drop(columns=features_to_drop)
X_test = X_test.drop(columns=features_to_drop)

print(f"\nКоличество удаленных признаков: {len(features_to_drop)}")

Столбцы в y_train: ['Risk']

Пары признаков с высокой корреляцией:
PARA_B - TOTAL: 0.994

Признаки, которые будут удалены:
PARA_B (корреляция с целевой: 0.155)

Количество удаленных признаков: 1


# 29

In [12]:
# Группируем данные по значению целевой переменной и вычисляем среднее PARA_A
means = X_train.groupby(y_train['Risk'])['PARA_A'].mean()

print("Средние значения PARA_A по классам:")
for class_label, mean_value in means.items():
    print(f"Класс {class_label}: {mean_value:.2f}")

print(f"\nМаксимальное среднее значение: {max(means):.2f}")

Средние значения PARA_A по классам:
Класс 0: 0.28
Класс 1: 4.16

Максимальное среднее значение: 4.16


# 30

In [13]:
# Фильтруем данные по условию Money_Value < 10 и вычисляем медиану TOTAL
median_total = X_train[X_train['Money_Value'] < 10]['TOTAL'].median()
print(f"Медиана TOTAL для объектов с Money_Value < 10: {median_total:.2f}")

Медиана TOTAL для объектов с Money_Value < 10: 1.08


In [14]:
X_train[X_train['Money_Value'] < 10]['TOTAL'].sort_values().median()

1.08

In [15]:
X_train[X_train['Money_Value'] < 10]['TOTAL'].sort_values()[210:230]

464    1.0600
224    1.0700
14     1.0700
241    1.0700
507    1.0800
445    1.0800
165    1.0900
307    1.1000
77     1.1000
498    1.1000
287    1.1000
283    1.1000
119    1.1000
424    1.1000
72     1.1106
22     1.1200
329    1.1200
56     1.1200
482    1.1300
136    1.1400
Name: TOTAL, dtype: float64

# 31

In [16]:
# Фильтруем объекты с Risk = 1 и PARA_A > 0.5
filtered_data = X_train[(y_train['Risk'] == 1) & (X_train['PARA_A'] > 0.5)]

# Проверяем, все ли значения Loss равны 0
all_zeros = (filtered_data['Loss'] == 0).all()

print(f"Все значения Loss равны 0: {all_zeros}")

# Для проверки выведем уникальные значения Loss
print("\nУникальные значения Loss:")
print(filtered_data['Loss'].unique())

Все значения Loss равны 0: False

Уникальные значения Loss:
[0 2 1]


# 32

In [17]:
# Вычисляем среднее значение Risk для локаций типа "A"
mean_risk_a = y_train[X_train['LOCATION_ID'] == 'A']['Risk'].mean()

# Вычисляем среднее значение Risk для всех остальных локаций
mean_risk_others = y_train[X_train['LOCATION_ID'] != 'A']['Risk'].mean()

print(f"Среднее значение Risk для локаций типа 'A': {mean_risk_a:.4f}")
print(f"Среднее значение Risk для остальных локаций: {mean_risk_others:.4f}")
print(f"\nRisk для типа 'A' больше, чем для остальных: {mean_risk_a > mean_risk_others}")

Среднее значение Risk для локаций типа 'A': 0.7273
Среднее значение Risk для остальных локаций: 0.5960

Risk для типа 'A' больше, чем для остальных: True


# 33

In [18]:
# Фильтруем данные по условиям Risk = 0 и numbers = 5.0
filtered_data = X_train[(y_train['Risk'] == 0) & (X_train['numbers'] == 5.0)]

# Вычисляем 75%-квантиль для столбца History с интерполяцией 'higher'
quantile_75 = filtered_data['History'].quantile(0.75, interpolation='higher')

print(f"75%-квантиль значений History: {int(quantile_75)}")

75%-квантиль значений History: 0


In [19]:
filtered_data['History'].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0])

# 34

In [20]:
# Применяем one-hot encoding к столбцу LOCATION_ID
X_train_encoded = pd.get_dummies(X_train, columns=['LOCATION_ID'])
X_test_encoded = pd.get_dummies(X_test, columns=['LOCATION_ID'])

# Убеждаемся, что в тестовых данных есть все категории из тренировочных
for col in X_train_encoded.columns:
    if col not in X_test_encoded.columns:
        X_test_encoded[col] = 0

# Приводим порядок столбцов в тестовых данных к порядку в тренировочных
X_test_encoded = X_test_encoded[X_train_encoded.columns]

# Получаем количество признаков после кодировки
n_features = X_train_encoded.shape[1]
print(f"Количество признаков после one-hot encoding: {n_features}")

Количество признаков после one-hot encoding: 11


In [21]:
X_train_encoded.head()

,PARA_A,TOTAL,numbers,Money_Value,Loss,History,LOCATION_ID_A,LOCATION_ID_B,LOCATION_ID_C,LOCATION_ID_D,LOCATION_ID_F
0,0.28,0.53,5.0,0.00,0,0,False,False,True,False,False
1,0.00,0.00,5.0,0.03,0,0,True,False,False,False,False
2,5.65,72.81,6.0,27.23,0,0,False,False,True,False,False
3,1.27,1.27,5.0,175.90,0,0,False,True,False,False,False
4,1.60,1.60,5.0,0.06,0,0,False,False,False,False,True


# 35

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Создаем модель с параметрами по умолчанию
knn = KNeighborsClassifier(
    n_neighbors=5,
    # weights='uniform',
    # algorithm='auto',
    # leaf_size=30,
    # p=2,
    # metric='minkowski',
    # metric_params=None, 
    # n_jobs=None
)

# Проводим кросс-валидацию на тренировочных данных
scores = cross_val_score(knn, X_train_encoded, y_train['Risk'], cv=3)

# Вычисляем среднее значение accuracy
mean_accuracy = scores.mean()

print(f"Среднее значение accuracy на кросс-валидации: {mean_accuracy:.2f}")

Среднее значение accuracy на кросс-валидации: 0.91


# 36

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Создаем базовую модель
knn = KNeighborsClassifier(
    # weights='uniform',
    # algorithm='auto',
    # leaf_size=30,
    # p=2,
    # metric='minkowski'
)

# Задаем сетку параметров для поиска
param_grid = {
    'n_neighbors': range(2, 21)  # от 2 до 20 включительно
}

# Создаем объект GridSearchCV
grid_search = GridSearchCV(
    knn,
    param_grid,
    cv=3,
    scoring='accuracy',
    verbose=1
)

# Выполняем поиск по сетке
grid_search.fit(X_train_encoded, y_train['Risk'])

# Получаем лучшее значение n_neighbors
best_n_neighbors = grid_search.best_params_['n_neighbors']
print(f"Оптимальное количество соседей: {best_n_neighbors}")

Fitting 3 folds for each of 19 candidates, totalling 57 fits
Оптимальное количество соседей: 5


# 37

In [24]:
# 1. Находим два признака с наибольшей корреляцией с таргетом
numeric_features = X_train_encoded.select_dtypes(include=['int64', 'float64']).columns
target_correlations = abs(X_train_encoded[numeric_features].corrwith(y_train['Risk']))
top_2_features = target_correlations.nlargest(2).index.tolist()

# 2. Создаем новый признак 'mult' в обоих наборах данных
X_train_encoded['mult'] = X_train_encoded[top_2_features[0]] * X_train_encoded[top_2_features[1]]
X_test_encoded['mult'] = X_test_encoded[top_2_features[0]] * X_test_encoded[top_2_features[1]]

# 3. Создаем и настраиваем GridSearchCV
knn = KNeighborsClassifier(
    weights='uniform',
    algorithm='auto',
    leaf_size=30,
    p=2,
    metric='minkowski'
)

param_grid = {
    'n_neighbors': range(2, 21)  # от 2 до 20 включительно
}

grid_search = GridSearchCV(
    knn,
    param_grid,
    cv=3,
    scoring='accuracy'
)

# 4. Выполняем поиск по сетке
grid_search.fit(X_train_encoded, y_train['Risk'])

# 5. Получаем лучшее значение n_neighbors
best_n_neighbors = grid_search.best_params_['n_neighbors']
print(f"Оптимальное количество соседей: {best_n_neighbors}")

Оптимальное количество соседей: 6


In [25]:
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
import numpy as np

# 1. Подготовка данных
# One-hot кодирование
X_train_encoded = pd.get_dummies(X_train, columns=['LOCATION_ID'])
X_test_encoded = pd.get_dummies(X_test, columns=['LOCATION_ID'])

# Убеждаемся, что в тестовых данных есть все категории из тренировочных
for col in X_train_encoded.columns:
    if col not in X_test_encoded.columns:
        X_test_encoded[col] = 0
X_test_encoded = X_test_encoded[X_train_encoded.columns]

# 2. Создание новых признаков
# Отношения
X_train_encoded['PARA_A_to_TOTAL'] = X_train_encoded['PARA_A'] / (X_train_encoded['TOTAL'] + 1e-8)
X_test_encoded['PARA_A_to_TOTAL'] = X_test_encoded['PARA_A'] / (X_test_encoded['TOTAL'] + 1e-8)

# Взаимодействия
X_train_encoded['Money_Value_X_History'] = X_train_encoded['Money_Value'] * X_train_encoded['History']
X_test_encoded['Money_Value_X_History'] = X_test_encoded['Money_Value'] * X_test_encoded['History']

# Квадратичные признаки для числовых переменных
numeric_cols = ['PARA_A', 'TOTAL', 'Money_Value']
for col in numeric_cols:
    X_train_encoded[f'{col}_squared'] = X_train_encoded[col] ** 2
    X_test_encoded[f'{col}_squared'] = X_test_encoded[col] ** 2

# 3. Масштабирование числовых признаков
scaler = StandardScaler()
numeric_features = X_train_encoded.select_dtypes(include=['float64', 'int64']).columns
X_train_encoded[numeric_features] = scaler.fit_transform(X_train_encoded[numeric_features])
X_test_encoded[numeric_features] = scaler.transform(X_test_encoded[numeric_features])

# 4. Обучение модели
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.03,
    depth=6,
    l2_leaf_reg=3,
    random_seed=42,
    verbose=False
)

# Обучаем модель
model.fit(X_train_encoded, y_train['Risk'])

# Получаем предсказания на тренировочном и тестовом наборах
train_pred = model.predict(X_train_encoded)
test_pred = model.predict(X_test_encoded)

# Оцениваем качество на тренировочном наборе
train_accuracy = (train_pred == y_train['Risk']).mean()
print(f"Accuracy на тренировочном наборе: {train_accuracy:.4f}")

# Сохраняем предсказания для тестового набора
predictions = pd.DataFrame({'predictions': test_pred})
predictions.to_csv('predictions.csv', index=False, header=False)

Accuracy на тренировочном наборе: 0.9908
